In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from distributed import Client

client = Client()
client

/Users/scott/anaconda3/envs/hyperband/lib/python3.7/site-packages/distributed/bokeh/core.py:55: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: tcp://127.0.0.1:64850 Dashboard: http://127.0.0.1:64851/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


In [3]:
import numpy as np
from dask_ml.model_selection._hyperband import PartialDataFit, HyperbandSearchCV
import umap
from sklearn.cluster import DBSCAN
from sklearn.datasets import load_digits
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [4]:
digits = load_digits()
X, y = digits.data, digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(X.shape)

params = {
    "umap__n_neighbors": range(5, 50),
    "umap__min_dist": np.logspace(-3, -0.3, num=1000),
    "umap__metric": ["l1", "l2", "cosine"],
    "umap__learning_rate": np.logspace(-1, 1, num=1000),
    "umap__init": ["spectral", "random"],
}
classifier = Pipeline(
    [("umap", umap.UMAP()), ("nearest", KNeighborsClassifier(n_neighbors=10))]
)

classifier.fit(X_train, y_train)
y_hat = classifier.predict(X_test)
base_error = (y_hat != y_test).sum() / len(y_hat)
base_error

(1797, 64)


0.028888888888888888

In [5]:
est = PartialDataFit(classifier)
search = HyperbandSearchCV(est, params, 27)
search.model

PartialDataFit(estimator=Pipeline(memory=None,
     steps=[('umap', UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
   learning_rate=1.0, local_connectivity=1.0, metric='euclidean',
   metric_kwds=None, min_dist=0.1, n_components=2, n_epochs=None,
   n_neighbors=15, negative_sample_rate=5, random_state=None,
   repulsion_strength=1.0, ...i',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform'))]),
        memory=None,
        steps=[('umap', UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
   learning_rate=1.0, local_connectivity=1.0, metric='euclidean',
   metric_kwds=None, min_dist=0.1, n_components=2, n_epochs=None,
   n_neighbors=15, negative_sample_rate=5, random_state=None,
   repulsion_strength=1.0, ...i',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform'))])

In [6]:
from sklearn.base import clone
kwargs = {'umap__n_neighbors': 27, 'umap__min_dist': 0.5011872336272722, 'umap__metric': 'l1', 'umap__learning_rate': 0.15072253093107563, 'umap__init': 'spectral'}

clone(est).set_params(**kwargs)  # test to make sure same type returned

PartialDataFit(estimator=Pipeline(memory=None,
     steps=[('umap', UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
   learning_rate=0.15072253093107563, local_connectivity=1.0, metric='l1',
   metric_kwds=None, min_dist=0.5011872336272722, n_components=2,
   n_epochs=None, n_neighbors=27, negative_sample_rate=5,
   random_state=None, ...i',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform'))]),
        memory=None,
        steps=[('umap', UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
   learning_rate=0.15072253093107563, local_connectivity=1.0, metric='l1',
   metric_kwds=None, min_dist=0.5011872336272722, n_components=2,
   n_epochs=None, n_neighbors=27, negative_sample_rate=5,
   random_state=None, ...i',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform'))])

In [7]:
search.fit(X, y)

HyperbandSearchCV(aggressiveness=3, max_iter=27,
         model=PartialDataFit(estimator=Pipeline(memory=None,
     steps=[('umap', UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
   learning_rate=1.0, local_connectivity=1.0, metric='euclidean',
   metric_kwds=None, min_dist=0.1, n_components=2, n_epochs=None,
   n_neighbors=15, negative_sam...',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform'))]),
         params={'umap__n_neighbors': range(5, 50), 'umap__min_dist': array([0.001  , 0.00101, ..., 0.49808, 0.50119]), 'umap__metric': ['l1', 'l2', 'cosine'], 'umap__learning_rate': array([ 0.1    ,  0.10046, ...,  9.95401, 10.     ]), 'umap__init': ['spectral', 'random']},
         patience=False, tol=0.001)

In [8]:
search.best_score_

0.95

In [9]:
search.best_estimator_

PartialDataFit(estimator=Pipeline(memory=None,
     steps=[('umap', UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
   learning_rate=0.3602106562357071, local_connectivity=1.0,
   metric='cosine', metric_kwds=None, min_dist=0.002607451362831879,
   n_components=2, n_epochs=None, n_neighbors=22, negative_sample_rate=5,
   random_state=N...i',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform'))]),
        memory=None,
        steps=[('umap', UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
   learning_rate=0.3602106562357071, local_connectivity=1.0,
   metric='cosine', metric_kwds=None, min_dist=0.002607451362831879,
   n_components=2, n_epochs=None, n_neighbors=22, negative_sample_rate=5,
   random_state=N...i',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform'))])

In [10]:
search.best_estimator_._calls  # test this is max_iter

9